In [ ]:
# GPU 런타임 설정을 한 후 실행한다.
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.5MB/s 
     |████████████████████████████████| 92kB 11.7MB/s 
     |████████████████████████████████| 460kB 57.5MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
# 패키지 설치
import konlpy
import pandas as pd
import numpy as np
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from sklearn import model_selection, metrics

# 토큰 파서
def twitter_tokenizer(text):
    return Twitter().morphs(text)

In [ ]:
twitter_tokenizer("Welcome to data science world!...한국말도 똑 같아요...")

/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['Welcome',
 'to',
 'data',
 'science',
 'world',
 '!...',
 '한국말',
 '도',
 '똑',
 '같아요',
 '...']

In [ ]:
!curl -L https://bit.ly/2X9Owwr -o ratings_train.txt
!curl -L https://bit.ly/2WuLd5I -o ratings_test.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   152  100   152    0     0   1504      0 --:--:-- --:--:-- --:--:--  1520
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 14.0M  100 14.0M    0     0  10.0M      0  0:00:01  0:00:01 --:--:-- 24.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   151  100   151    0     0   2220      0 --:--:-- --:--:-- --:--:--  2220
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 4827k  100 4827k    0     0  4230k      0  0:00:01  0:00:01 --:--:-- 4230k


In [ ]:
# 데이터 로드
df_train = pd.read_csv('ratings_train.txt', delimiter='\t', keep_default_na=False)
df_test = pd.read_csv('ratings_test.txt', delimiter='\t', keep_default_na=False)


In [ ]:
df_train[:3]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [ ]:
df_train['document'].values == df_train['document'].to_numpy()

array([ True,  True,  True, ...,  True,  True,  True])

In [ ]:
text_train, y_train = df_train['document'].to_numpy(), df_train['label'].to_numpy()
text_test, y_test = df_test['document'].to_numpy(), df_test['label'].to_numpy()

In [ ]:
text_train.shape, y_train.shape, text_test.shape, y_test.shape

((150000,), (150000,), (50000,), (50000,))

In [ ]:
# too much... -> let's take few of them
text_train, y_train = text_train[:2000], y_train[:2000]
text_test, y_test = text_test[:1000], y_test[:1000]

In [ ]:
cv = TfidfVectorizer(tokenizer=twitter_tokenizer, max_features=3000)
X_train = cv.fit_transform(text_train)
X_test = cv.transform(text_test)

/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((2000, 3000), (2000,), (1000, 3000), (1000,))

In [ ]:
X_train[0][-10:].toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
y_test[:3]

array([1, 0, 0])

In [ ]:
cv.get_feature_names()[-5:]

['힘들건데', '힘들게', '힘들다', '힘들었네요', '힘들었음']

# 로지스틱 회귀

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
result = lr.fit(X_train,y_train)
feature_names = cv.get_feature_names()
print("Training : ", result.score(X_train, y_train))
print("Testing : ", result.score(X_test, y_test))

Training :  0.916
Testing :  0.771


# MLP

In [ ]:
# use one-hot encoded target
from keras.utils import to_categorical
y_train_ohe = to_categorical(y_train)
y_test_ohe = to_categorical(y_test)
max_words = X_train.shape[1]
batch_size = 100
nb_epoch = 5

model = Sequential()
model.add(Dense(64, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train.toarray(), y_train_ohe, epochs=nb_epoch, 
          batch_size=batch_size) 
print("Training : ", model.evaluate(X_train.toarray(), y_train_ohe))
print("Testing : ", model.evaluate(X_test.toarray(), y_test_ohe))

Epoch 1/5
20/20 [==============================] - 0s 3ms/step - loss: 0.6896 - accuracy: 0.5435
Epoch 2/5
20/20 [==============================] - 0s 2ms/step - loss: 0.6449 - accuracy: 0.8175
Epoch 3/5
20/20 [==============================] - 0s 2ms/step - loss: 0.5074 - accuracy: 0.8765
Epoch 4/5
20/20 [==============================] - 0s 3ms/step - loss: 0.3116 - accuracy: 0.9065
Epoch 5/5
63/63 [==============================] - 0s 2ms/step - loss: 0.1218 - accuracy: 0.9700
Training :  [0.12181834876537323, 0.9700000286102295]
32/32 [==============================] - 0s 2ms/step - loss: 0.5240 - accuracy: 0.7530
Testing :  [0.524045467376709, 0.753000020980835]


In [ ]:
# use binary target
from keras.utils import to_categorical

max_words = X_train.shape[1]
batch_size = 100
nb_epoch = 5

model = Sequential()
model.add(Dense(64, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train.toarray(), y_train, epochs=nb_epoch, 
          batch_size=batch_size) 
print("Training : ", model.evaluate(X_train.toarray(), y_train))
print("Testing : ", model.evaluate(X_test.toarray(), y_test))

Epoch 1/5
20/20 [==============================] - 0s 3ms/step - loss: 0.6905 - accuracy: 0.5420
Epoch 2/5
20/20 [==============================] - 0s 2ms/step - loss: 0.6560 - accuracy: 0.8070
Epoch 3/5
20/20 [==============================] - 0s 2ms/step - loss: 0.5469 - accuracy: 0.8690
Epoch 4/5
20/20 [==============================] - 0s 2ms/step - loss: 0.3662 - accuracy: 0.9005
Epoch 5/5
63/63 [==============================] - 0s 2ms/step - loss: 0.1608 - accuracy: 0.9580
Training :  [0.1608031690120697, 0.9580000042915344]
32/32 [==============================] - 0s 2ms/step - loss: 0.4971 - accuracy: 0.7710
Testing :  [0.49714699387550354, 0.7710000276565552]


# RNN

In [ ]:
# just for checking
X_train.A[0] == X_train.toarray()[0]

array([ True,  True,  True, ...,  True,  True,  True])

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2000, 3000), (1000, 3000), (2000,), (1000,))

In [ ]:
# RNN 학습을 위한 데이터 재배열
X_train_rnn = X_train.A.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn = X_test.A.reshape((X_test.shape[0], 1, X_test.shape[1]))

print(X_train_rnn.shape, X_test_rnn.shape)

(2000, 1, 3000) (1000, 1, 3000)


In [ ]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense

model = Sequential()
model.add(SimpleRNN(128, 
                    input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), 
                    return_sequences=True))
# return_sequences: return the last output in the output sequence, or the full sequence
# By this, it is possible to access the hidden state output for each input time step.
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(SimpleRNN(128))
model.add(Dropout(0.2))
model.add(Dense(2, activation="softmax"))
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])   

In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 1, 128)            400512    
_________________________________________________________________
activation_7 (Activation)    (None, 1, 128)            0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 1, 128)            0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 128)               32896     
_________________________________________________________________
dropout_15 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 2)                 258       
Total params: 433,666
Trainable params: 433,666
Non-trainable params: 0
________________________________________________

In [ ]:
model.fit(X_train_rnn, y_train_ohe, batch_size = 100,
          epochs=nb_epoch)

Epoch 1/5
20/20 [==============================] - 0s 4ms/step - loss: 0.6861 - accuracy: 0.5955
Epoch 2/5
20/20 [==============================] - 0s 4ms/step - loss: 0.5878 - accuracy: 0.8435
Epoch 3/5
20/20 [==============================] - 0s 4ms/step - loss: 0.3478 - accuracy: 0.8980
Epoch 4/5
20/20 [==============================] - 0s 4ms/step - loss: 0.1680 - accuracy: 0.9460
Epoch 5/5
20/20 [==============================] - 0s 4ms/step - loss: 0.0852 - accuracy: 0.9790


In [ ]:
y_pred = np.argmax(model.predict(X_test_rnn), axis=1)
print("accuracy score: ", metrics.accuracy_score(y_test, y_pred))
print("Training : ", model.evaluate(X_train_rnn, y_train_ohe))
print("Testing : ", model.evaluate(X_test_rnn, y_test_ohe))

accuracy score:  0.756
63/63 [==============================] - 0s 3ms/step - loss: 0.0518 - accuracy: 0.9880
Training :  [0.05180356651544571, 0.9879999756813049]
32/32 [==============================] - 0s 3ms/step - loss: 0.6205 - accuracy: 0.7560
Testing :  [0.6205360889434814, 0.7559999823570251]


# LSTM
- https://colah.github.io/posts/2015-08-Understanding-LSTMs/

In [ ]:
from keras.layers import LSTM
model = Sequential()
model.add(LSTM(128, 
               input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), 
               return_sequences=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train_rnn, y_train_ohe, batch_size = 100,
          epochs=nb_epoch)

Epoch 1/5
20/20 [==============================] - 0s 5ms/step - loss: 0.6929 - accuracy: 0.5070
Epoch 2/5
20/20 [==============================] - 0s 5ms/step - loss: 0.6865 - accuracy: 0.5930
Epoch 3/5
20/20 [==============================] - 0s 4ms/step - loss: 0.6441 - accuracy: 0.8170
Epoch 4/5
20/20 [==============================] - 0s 4ms/step - loss: 0.4962 - accuracy: 0.8760
Epoch 5/5
20/20 [==============================] - 0s 4ms/step - loss: 0.2970 - accuracy: 0.9050


In [ ]:
y_pred = np.argmax(model.predict(X_test_rnn), axis=1)
print("accuracy score: ", metrics.accuracy_score(y_test, y_pred))
print("Training : ", model.evaluate(X_train_rnn, y_train_ohe))
print("Testing : ", model.evaluate(X_test_rnn, y_test_ohe))

accuracy score:  0.773
63/63 [==============================] - 0s 3ms/step - loss: 0.2005 - accuracy: 0.9340
Training :  [0.2004823386669159, 0.9340000152587891]
32/32 [==============================] - 0s 3ms/step - loss: 0.5029 - accuracy: 0.7730
Testing :  [0.5029308199882507, 0.7730000019073486]


# GRU

In [ ]:
from keras.layers import GRU
model = Sequential()
model.add(GRU(128, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), return_sequences=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(GRU(128))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train_rnn, y_train_ohe, batch_size = 100,
          epochs=nb_epoch)

Epoch 1/5
20/20 [==============================] - 0s 4ms/step - loss: 0.6922 - accuracy: 0.4990
Epoch 2/5
20/20 [==============================] - 0s 4ms/step - loss: 0.6619 - accuracy: 0.7740
Epoch 3/5
20/20 [==============================] - 0s 4ms/step - loss: 0.5127 - accuracy: 0.8720
Epoch 4/5
20/20 [==============================] - 0s 4ms/step - loss: 0.2842 - accuracy: 0.9055
Epoch 5/5
20/20 [==============================] - 0s 4ms/step - loss: 0.1560 - accuracy: 0.9460


In [ ]:
y_pred = np.argmax(model.predict(X_test_rnn), axis=1)
print("accuracy score: ", metrics.accuracy_score(y_test, y_pred))
print("Training : ", model.evaluate(X_train_rnn, y_train_ohe))
print("Testing : ", model.evaluate(X_test_rnn, y_test_ohe))

accuracy score:  0.757
63/63 [==============================] - 0s 3ms/step - loss: 0.1007 - accuracy: 0.9740
Training :  [0.10072411596775055, 0.9739999771118164]
32/32 [==============================] - 0s 3ms/step - loss: 0.5841 - accuracy: 0.7570
Testing :  [0.5841045379638672, 0.7570000290870667]
